Wildfire season started in Washington state in April this year.

See [Mandel et al. (2008)](https://doi.org/10.1016/j.matcom.2008.03.015) and [Reisch et al. (2024)](https://doi.org/10.1016/j.camwa.2024.01.024).

In [ ]:
import firedrake
from firedrake import Constant, inner, grad, dx, exp
import irksome
from irksome import Dt

nx, ny = 32, 32
mesh = firedrake.UnitSquareMesh(nx, ny, diagonal="crossed")
cg = firedrake.FiniteElement("CG", "triangle", 1)
function_space = firedrake.FunctionSpace(mesh, cg)

The reaction rate depends on two parameters, a critical temperature $T_c$ and a temperature scale $\Delta T$.
The rate is zero below the critical temperature and smoothly approaches 1 above.
The temperature scale dictates how fast the reaction rate approaches its maximum.
The explicit form of the rate is
$$R(T) = \exp\left(-\frac{\Delta T}{T - T_c}\right).$$
If the temperature exceeds the critical temperature by a factor of, say, $5\Delta T$, then the rate is more or less indistinguishable from its maximum value of 1.

We want to be careful about the numerics when we implement this equation.
Suppose the temperature is equal to $T_c + \delta T$ where $\delta T$ is very small.
In effect we're then trying to evaluate $\exp(-1 / \epsilon)$.
Depending on how the exponential function is implemented, either this or its derivative could become an unholy mess.

In [ ]:
T_c = Constant(100.0)
ΔT = Constant(100.0)

def reaction_rate(T):
    return firedrake.conditional(T <= T_c, 0, exp(-ΔT / (T - T_c)))

In [ ]:
x = firedrake.SpatialCoordinate(mesh)
T_a = Constant(15.0)
f = Constant(2.0)
δT = T_c + f * ΔT - T_a
expr = T_a + f * δT * x[0]
T = firedrake.Function(function_space).interpolate(expr)

In [ ]:
import matplotlib.pyplot as plt

r = firedrake.Function(function_space).interpolate(reaction_rate(T))
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(r, axes=ax, cmap="inferno")
fig.colorbar(colors);

The other coefficients we need are the heat released per unit mass of fuel $Q$, the consumption rate $C$ of fuel, the ambient air temperature $T_a$, and the exchange coefficient $H$ with the upper atmosphere.

In [ ]:
T = firedrake.Function(Q)
F = firedrake.Function(Q)

T_a = Constant(15.0)

G_transport = (Dt(T) + inner(u, grad(T))) * ϕ * dx
G_diffusion = k * inner(grad(T), grad(ϕ)) * dx
G_reaction = (Q * R(T) * F - H * (T - T_a)) * ϕ * dx
G_burning = (Dt(F) + C * R(T) * F) * ψ * dx

G = G_transport + G_diffusion - G_reaction + G_burning